<a href="https://colab.research.google.com/github/minyeseul201701330/hufs/blob/main/%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [ ]:
# konlpy(한국어형태소분석기)를 실행하기
import konlpy

ModuleNotFoundError: ignored

In [ ]:
# konlpy 설치하기 ## https://data1000.tistory.com/33
!pip3 install jpype1==0.7.0
!pip3 install konlpy

In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# 실습을 위한 영화 리뷰: 남산의 부장들, 다만 악에서 구하소서, 반도
## https://drive.google.com/open?id=1CONG4YCBYcmYcNgsy1XEL8Lb2IgwsS_O

rawdata_downloaded = drive.CreateFile({'id': '1CONG4YCBYcmYcNgsy1XEL8Lb2IgwsS_O'})
rawdata_downloaded.GetContentFile('rawdata.txt')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata.txt', sep="\t")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,no,movie,nickname,text,score,like,dislike,time,watch_movie
0,1,남산의 부장들,erlo****,김재규를 미화하지않고객관적이며 인간적으로 묘사했던게 좋았고무엇보다 머머리를 좀도둑놈...,10,5989,1987,2020-01-23 14:55,관람객
1,2,남산의 부장들,빛그림(namj****),연기 잘 하는 배우가 모여있습니다,10,4918,1417,2020-01-22 15:55,관람객
2,3,남산의 부장들,쩡(jeon****),이 영화는 이병헌 연기의 최정점을 볼 수 있다ㄷㄷ 한 장면 한 장면 긴장감을 놓을 ...,10,4385,1678,2020-01-22 10:10,관람객
3,4,남산의 부장들,Joonseosa(myeo****),오랜만에 볼만한 한국영화!이성민 배우님 박통 연기 소름돋았습니다,10,3625,1227,2020-01-22 20:29,관람객
4,5,남산의 부장들,ㅎㅎㅎ(eorm****),다 좋은데 마지막에 뭐냐; 전두환이 돈 챙겨가는거ㅋㅋㅋ개찐빠네 마지막이,10,3231,952,2020-01-22 11:19,관람객
...,...,...,...,...,...,...,...,...,...
59711,59712,반도,김수빈(eken****),기분이 반도하다. 꼭봐라 이기분 다들 느껴보길 바란다,1,465,1827,2020-07-15 19:17,관람객
59712,59713,반도,강맥스(khkg****),코로나는 피했는데 이걸 못 피하네...,1,586,2084,2020-07-17 0:59,관람객
59713,59714,반도,이라지마이소(chlg****),쌉유치합니다 대사부터... 후반부가 다 예상이 되는전개이고 열심히만든건 ㅇㅈ하...,2,600,2106,2020-07-15 18:49,관람객
59714,59715,반도,리얼리스트(wkdd****),"1)영화시작부터 신파가 시작되고, 총 4번의 신파가 나오는데 전부 1도 슬프지 않다...",4,368,1885,2020-07-15 21:52,관람객


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[3] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('처음', 'Noun'),
 ('부터', 'Josa'),
 ('엔딩', 'Noun'),
 ('까지', 'Josa'),
 ('신파', 'Noun'),
 ('연속', 'Noun'),
 ('에', 'Josa'),
 ('앞뒤', 'Noun'),
 ('가', 'Josa'),
 ('안', 'VerbPrefix'),
 ('맞는', 'Verb'),
 ('대사', 'Noun'),
 ('와', 'Josa'),
 ('연출', 'Noun'),
 ('에', 'Josa'),
 ('그냥', 'Noun'),
 ('개', 'Noun'),
 ('하급', 'Noun'),
 ('B', 'Alpha'),
 ('급', 'Noun'),
 ('영화', 'Noun'),
 ('임', 'Noun'),
 ('돈', 'Noun'),
 ('쳐', 'Verb'),
 ('바른', 'Modifier'),
 ('느낌', 'Noun'),
 ('은', 'Josa'),
 ('나서', 'Verb'),
 ('별', 'Modifier'),
 ('점', 'Noun'),
 ('1', 'Number'),
 ('점', 'Noun'),
 ('줄려다가', 'Verb'),
 ('2', 'Number'),
 ('점', 'Noun'),
 ('준다', 'Verb')]

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,처음,Noun,반도,1
1,부터,Josa,반도,1
2,엔딩,Noun,반도,1
3,까지,Josa,반도,1
4,신파,Noun,반도,1
5,연속,Noun,반도,1
6,에,Josa,반도,1
7,앞뒤,Noun,반도,1
8,가,Josa,반도,1
9,안,VerbPrefix,반도,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[      0            1     type  count
 0   김재규         Noun  남산의 부장들      1
 1     를         Josa  남산의 부장들      1
 2    미화         Noun  남산의 부장들      1
 3    하지         Verb  남산의 부장들      1
 4    않고         Verb  남산의 부장들      1
 5    객관         Noun  남산의 부장들      1
 6     적       Suffix  남산의 부장들      1
 7    이며         Josa  남산의 부장들      1
 8    인간         Noun  남산의 부장들      1
 9     적       Suffix  남산의 부장들      1
 10   으로         Josa  남산의 부장들      1
 11   묘사         Noun  남산의 부장들      1
 12  했던게         Verb  남산의 부장들      1
 13  좋았고    Adjective  남산의 부장들      1
 14   무엇         Noun  남산의 부장들      1
 15   보다         Josa  남산의 부장들      1
 16    머         Noun  남산의 부장들      1
 17   머리         Noun  남산의 부장들      1
 18    를         Josa  남산의 부장들      1
 19  좀도둑         Noun  남산의 부장들      1
 20    놈         Noun  남산의 부장들      1
 21  시키로         Verb  남산의 부장들      1
 22   그려         Verb  남산의 부장들      1
 23  줬던게         Verb  남산의 부장들      1
 24   최고         Noun  남산의 부장들      1
 25    의    

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,김재규,Noun,남산의 부장들,1
1,를,Josa,남산의 부장들,1
2,미화,Noun,남산의 부장들,1
3,하지,Verb,남산의 부장들,1
4,않고,Verb,남산의 부장들,1
...,...,...,...,...
31,점,Noun,반도,1
32,줄려다가,Verb,반도,1
33,2,Number,반도,1
34,점,Noun,반도,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,김재규,Noun,남산의 부장들,1
1,를,Josa,남산의 부장들,1
2,미화,Noun,남산의 부장들,1
3,하지,Verb,남산의 부장들,1
4,않고,Verb,남산의 부장들,1
...,...,...,...,...
31,점,Noun,반도,1
32,줄려다가,Verb,반도,1
33,2,Number,반도,1
34,점,Noun,반도,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소  품사           분류         
    Foreign      남산의 부장들           2
                  다만 악에서 구하소서       1
!    Foreign      다만 악에서 구하소서       1
     Punctuation  남산의 부장들         847
                  다만 악에서 구하소서    1353
                                 ... 
힝구   Noun         반도                1
힝힝   Noun         반도                1
８    Foreign      반도                1
；；   Foreign      다만 악에서 구하소서       1
～    Foreign      반도                1
Name: 카운트, Length: 82570, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소 품사          분류               
   Foreign     남산의 부장들         2
                다만 악에서 구하소서     1
!   Foreign     다만 악에서 구하소서     1
    Punctuation 남산의 부장들       847
                다만 악에서 구하소서  1353
...                           ...
힝구  Noun        반도              1
힝힝  Noun        반도              1
８   Foreign     반도              1
；；  Foreign     다만 악에서 구하소서     1
～   Foreign     반도              1

[82570 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')